In [1]:
from bs4 import BeautifulSoup
from random import randint, shuffle
from time import sleep
import re
import pandas as pd
import json
import math
import requests
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import os
from tqdm import tqdm_notebook as tqdm
import shutil
import numpy as np


pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", -1)
pd.set_option("display.float_format", "{:,.2f}".format)
pd.set_option("display.max_info_rows", 200)

In [2]:
df_products_detailed = pd.read_csv("products_detailed.csv")
df_products_detailed.head()

,id,name,short_desc,long_desc,item,item_name,list_price,variation,variants,price_low,price_high,is_limited_edition,is_natural_organic,is_natural_sephora,rating,review_count,user_favorites,categories,url,image_url,ingredients,suggested_usage,brand_name,brand_id,brand_long
0,P448857,Apple Cider Vinegar Detangling Conditioner,"An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine.","What it is: An ultra-lightweight conditioner that gently detangles hair and uses the power of apple cider vinegar to seal the hair’s cuticle, enhancing softness and shine. Hair Type: Straight, Wavy, Curly, and Coily Hair Texture: Fine, Medium, and Thick Key Benefits: Detangles, Adds Shine, and Prevents Damage Formulation: Liquid Highlighted Ingredients: - Sunflower Seed Extract: Protects hair color. - Argan Oil, Aloe Vera, and Lavender Extract: Add softness and shine. Ingredient Callouts: Free of sulfates SLS and SLES, parabens, and phthalates. This product is also vegan, cruelty-free, and gluten-free, and comes in recyclable packaging. What Else You Need to Know: This conditioner is formulated to detangle hair and provide an extra conditioning boost after cleansing hair. It is infused with conditioning agents argan oil, aloe vera, and lavender extract to nurture hair and sunflower seed extract to protect hair color.",2276269,6.5 oz/ 192 mL,30.00,None,NaN,30.00,30.00,0,0,0,2.80,5,280,"{'cat60131': 'Conditioner', 'cat1230050': 'Shampoo & Conditioner', 'cat130038': 'Hair'}",https://www.sephora.com/product/apple-cider-vinegar-detangling-conditioner-P448857?skuId=2276269,https://www.sephora.com/productimages/sku/s2276269-main-zoom.jpg,"-Sunflower Seed Extract: Protects hair color. -Argan Oil, Aloe Vera, and Lavender Extract: Add softness and shine. Water/Aqua/Eau, Cetearyl Alcohol, Caprylic/Capric Triglyceride, Behentrimonium Chloride, Cetrimonium Chloride, Stearalkonium Chloride, Diheptyl Succinate, Capryloyl Glycerin/Sebacic Acid Copolymer, Polyquaternium-47, Hydrolyzed Vegetable Protein, Apple Cider Vinegar, Glycerin, Hydroxyethylcellulose, Argania Spinosa Kernel Oil, Macadamia Ternifolia Seed Oil, Spathodea Campanulata Flower Extract, Taraxacum Officinale (Dandelion) Leaf Extract, Lavandula Angustifolia (Lavender) Extract, Aloe Barbadensis Leaf Juice, Tocopheryl Acetate, Butylene Glycol, Helianthus Annuus (Sunflower) Seed Extract, Tetrasodium Glutamate Diacetate, Sodium Hydroxide, Phenoxyethanol, Ethylhexylglycerin, Fragrance (Parfum).","Suggested Usage: -Apply to clean, wet hair. -Massage through mid-lengths and ends of hair, then rinse thoroughly. -For best results, cleanse before with dpHUE Apple Cider Vinegar Hair Rinse.",dpHUE,6147,NaN
1,P425464,Surf Spray Mini,"A saltwater-based styling spray that adds texture, volume, and hold for matte waves that mimic a fresh-off-the-beach look.","What it is: A saltwater-based styling spray that adds texture, volume, and hold for matte waves that mimic a fresh-off-the-beach look. Hair Type: Straight and Wavy Hair Texture: Fine and Medium Hair Concerns: Hold and Volumizing Formulation: Spray Highlighted Ingredients: - Ascophyllum Nudosum (Seaweed) Extract: Extracted from naturally occurring seaweed; softens and smoothes skin and moisturizes hair, increasing its elasticity, luster, and softness; and helps decrease hair’s static charge, thus giving more body and less flyaways. - Macrocystis Pyrifera (Kelp) Extract: A natural seaweed extract with a high iodine and mineral content; contributes shine and manageability to hair and reduces the harshness of some chemicals; the nutritive value of this extract makes it ideal for skin preparations. - Magnesium Sulfate: Commonly known as Epsom salts; a natural mineral, which imparts astringent action. Ingredient Callouts: Free of parabens, formaldehydes, phthalates, and mineral oil. This product is also cruelty-free. What Else You Need to Know: This saltwater-based, st

In [3]:
image_folder = "/Users/valmadrid/DataScienceBootcamp/Projects/Gift Recommendation/Gift-Recommendation/image/"
if not os.path.exists(image_folder):
    os.mkdir(image_folder)

In [6]:
def get_image(urls, filenames, folder_path):
    if len(urls)!=len(filenames):
        print("Error: urls and filenames list should have the same length")
        return
    for url, filename in zip(urls, filenames):
        if os.path.isfile(folder_path+filename):
            print("{} file already exists".format(filename))
            continue
        else:
            r = requests.get(url, allow_redirects=True)
            if r.status_code==200:
                open(folder_path+filename, 'wb').write(r.content)
            else:
                print("Error: opening image url for product {}".format(filename.replace(".png", "")))
            sleep(randint(3,5))

In [ ]:
urls = df_products_detailed.image_url.values
filenames = [filename+".png" for filename in df_products_detailed.id.values]
get_image(urls, filenames, image_folder)
r.close()

P448857.png file already exists
P425464.png file already exists
P414293.png file already exists
P377493.png file already exists
P425872.png file already exists
P380552.png file already exists
P454931.png file already exists
P394141.png file already exists
P449600.png file already exists
P406611.png file already exists
P440972.png file already exists
P459483.png file already exists
P296834.png file already exists
P384792.png file already exists
P393407.png file already exists
P443574.png file already exists
P425608.png file already exists
P280519.png file already exists
P434019.png file already exists
P446633.png file already exists
P9576.png file already exists
P404913.png file already exists
P439943.png file already exists
P444989.png file already exists
P395212.png file already exists
P395642.png file already exists
P449342.png file already exists
P425433.png file already exists
P122750.png file already exists
P440925.png file already exists
P428714.png file already exists
P431535.pn

P416365.png file already exists
P448565.png file already exists
P451744.png file already exists
P455880.png file already exists
P457014.png file already exists
P388768.png file already exists
P193706.png file already exists
P163102.png file already exists
P432401.png file already exists
P256833.png file already exists
P433139.png file already exists
P447782.png file already exists
P395766.png file already exists
P432728.png file already exists
P438630.png file already exists
P407173.png file already exists
P407489.png file already exists
P444141.png file already exists
P448861.png file already exists
P415621.png file already exists
P411398.png file already exists
P414030.png file already exists
P411232.png file already exists
P440682.png file already exists
P457844.png file already exists
P9940.png file already exists
P432417.png file already exists
P433656.png file already exists
P449159.png file already exists
P453807.png file already exists
P446886.png file already exists
P450051.pn